# Syntax for computing with components

In [1]:
from modeling.compute import Var
from modeling.gen2.execution import Component

## Component handles unit conversions

In [2]:
Pcomms = Var('P_{comms}', unit='W')
D = Var('D', unit='m')
h = Var('h', unit='km')
R = Var('R', unit='kW')
sizing_relationship = Pcomms*D**2/h**2*1e18/(8e6)

In [3]:
sizing_relationship

125000000000.0*D**2*P_{comms}/h**2

In [4]:
component = Component.fromsympy(sizing_relationship, tovar=R)

## Component attributes

In [5]:
component.inputs, component.indims

(('h', 'P_{comms}', 'D'), (1, 1, 1))

In [6]:
component.outputs, component.outdims

(('R',), (1,))

## Evaluate components

In [7]:
component.evaldict({'h':400, 'P_{comms}':10, 'D':10})

[DeviceArray(0.78125, dtype=float32)]

## Evaluate gradients through automatic differentiation

In [8]:
component.graddict({'h':400, 'P_{comms}':10, 'D':10})

{('R', 'h'): DeviceArray(-0.00390625, dtype=float32),
 ('R', 'P_{comms}'): DeviceArray(0.07812501, dtype=float32),
 ('R', 'D'): DeviceArray(0.15625, dtype=float32)}

## Component manual creation
Demonstration with matrix/vector quantities

In [11]:
import numpy as np
# fix numpy seed for reproducibility
np.random.seed(0)
A = np.random.rand(10,10)
b0 = np.random.rand(10)
x0 = np.random.rand(10)
fx = lambda x,b: (b@(A@x-b),) # The @ is numpys matrix multiplication
component = Component(fx, inputs=('x','b'), outputs=('y',), indims=(10,10), outdims=(1,))

In [12]:
component.evaldict({'x':x0, 'b':b0})

(14.399744393772462,)

In [13]:
component.graddict({'x':x0, 'b':b0})

{('y',
  'x'): DeviceArray([3.005095 , 3.383307 , 3.1705244, 3.0786974, 1.6391404,
              2.260299 , 2.7484593, 2.4617856, 4.085971 , 2.0236309],            dtype=float32),
 ('y',
  'b'): DeviceArray([2.698821  , 2.8170767 , 1.9712472 , 1.7998033 , 2.137879  ,
              1.732636  , 1.3720064 , 1.8314322 , 2.5308552 , 0.67787457],            dtype=float32)}

### Check against analytical derivatives

In [14]:
b0@A # dy/dx

array([3.00509536, 3.38330726, 3.17052427, 3.07869741, 1.63914044,
       2.26029913, 2.74845931, 2.46178547, 4.0859708 , 2.02363065])

In [15]:
A@x0-2*b0 # dy/db

array([2.69882065, 2.817077  , 1.97124716, 1.79980327, 2.137879  ,
       1.73263593, 1.37200649, 1.83143236, 2.53085497, 0.6778746 ])